In [46]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder,StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,classification_report
import tensorflow as tf
from scikeras.wrappers import KerasClassifier

In [47]:
train_data=pd.read_csv("../dataset/train_data.csv")
test_data=pd.read_csv("../dataset/test_data.csv")

In [48]:
print(f"Shape of the Training data:{train_data.shape}")
print(f"Shape of the Testing data:{test_data.shape}")

Shape of the Training data:(1550, 10)
Shape of the Testing data:(388, 10)


In [49]:
train_data=train_data=train_data.drop(["Unnamed: 0"],axis=1)
test_data=test_data.drop(["Unnamed: 0"],axis=1)

In [50]:
##split train data into the X_train and Y_train
X_train=train_data.drop(["delivery_status"],axis=1)
Y_train=train_data["delivery_status"]

In [51]:
##Adjust the datatype of  assembly_service_requested column
X_train["assembly_service_requested"]=X_train["assembly_service_requested"].astype("object")

In [52]:
# ### Numbercal columns 
numerical_features=X_train.select_dtypes(include="number").columns.to_list()
categorical_features=X_train.select_dtypes(include="object").columns.to_list()

In [53]:
### Define pipelines
nominal_pipeline=Pipeline(steps=[
    ("One-Hot-Encoder",OneHotEncoder(sparse_output=False,handle_unknown="ignore"))
])
numerical_pipeline=Pipeline(steps=[
    ("Standard Scaler",StandardScaler())
])

transfomer=ColumnTransformer(transformers=[
    ("Numerical Pipeline",numerical_pipeline,numerical_features),
    ("Nominal Pipeline",nominal_pipeline,categorical_features)
])

final_pipeline=Pipeline(steps=[
    ("Transfomer",transfomer)
  
])

In [54]:
X_train_preprocessed = final_pipeline.named_steps["Transfomer"].fit_transform(X_train)

# Get the ColumnTransformer from the pipeline
preprocessor = final_pipeline.named_steps["Transfomer"]

# Get feature names from the ColumnTransformer
feature_names = preprocessor.get_feature_names_out()
X_train_preprocessed=pd.DataFrame(X_train_preprocessed,columns=feature_names)

In [55]:
##split train data into the X_train and Y_train
X_test=test_data.drop(["delivery_status"],axis=1)
Y_test=test_data["delivery_status"]

##Adjust the datatype of  assembly_service_requested column
X_test["assembly_service_requested"]=X_test["assembly_service_requested"].astype("object")

X_test_preprocessed = final_pipeline.named_steps["Transfomer"].transform(X_test)
X_test_preprocessed=pd.DataFrame(X_test_preprocessed,columns=feature_names)

In [57]:
##mapping y variable with the numeric values
y_map={"Failed Delivery":0,"On Going":1,"Delivered":2}
Y_train=Y_train.map(y_map)
Y_test=Y_test.map(y_map)

In [60]:
##implementing ANN using tensorflow
model=tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_preprocessed.shape[1],)), ##Input Layer
    tf.keras.layers.Dense(10,activation="relu"), ## First Hidden Layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(6,activation="relu"), ##Second Hidden Layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(8,activation="relu"), ##Third Hidden Layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(3,activation="softmax") ##Outpur Layer
])

model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.fit(X_train_preprocessed,Y_train,validation_data=(X_test_preprocessed,Y_test),epochs=200,batch_size=20,verbose=1)

Epoch 1/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.3155 - loss: 1.4585 - val_accuracy: 0.3943 - val_loss: 1.0758
Epoch 2/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3871 - loss: 1.2687 - val_accuracy: 0.4716 - val_loss: 1.0529
Epoch 3/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4226 - loss: 1.2271 - val_accuracy: 0.4794 - val_loss: 1.0435
Epoch 4/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4439 - loss: 1.1782 - val_accuracy: 0.4871 - val_loss: 1.0362
Epoch 5/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4555 - loss: 1.1450 - val_accuracy: 0.4897 - val_loss: 1.0336
Epoch 6/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4626 - loss: 1.1074 - val_accuracy: 0.4897 - val_loss: 1.0315
Epoch 7/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4626 - loss: 1.0949 - val_accuracy: 0.4897 - val_loss: 1.0295
Epoch 8/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4606 - loss: 1.1036 - val_accuracy: 0.4871 - v

In [61]:
y_pred=model.predict(X_test_preprocessed)
y_pred

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


array([[0.5066513 , 0.31862986, 0.17471886],
       [0.5006082 , 0.32450894, 0.17488284],
       [0.498116  , 0.32716662, 0.17471744],
       ...,
       [0.5009123 , 0.32406732, 0.1750203 ],
       [0.5006968 , 0.3242281 , 0.17507516],
       [0.5004609 , 0.32447684, 0.17506221]],
      shape=(388, 3), dtype=float32)

In [62]:
y_pred= np.argmax(y_pred, axis=1)
accuracy_score(y_pred,Y_test)

0.4948453608247423

In [63]:
y_pred_train=model.predict(X_train_preprocessed)
y_pred_train= np.argmax(y_pred_train, axis=1)
accuracy_score(y_pred_train,Y_train)

 1/49 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


0.49483870967741933